In [1]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, DigitalDriveType

In [ ]:
ao_task = nidaqmx.Task()
ai_task = nidaqmx.Task()


In [ ]:
ao_task.ao_channels.add_ao_voltage_chan("NI_DAQ/ao1", name_to_assign_to_channel="SetPointInput", min_val=0, max_val=5)
ai_task.ai_channels.add_ai_voltage_chan("NI_DAQ/ai3", name_to_assign_to_channel="ValveTestPoint", min_val=0, max_val=5, terminal_config=nidaqmx.constants.TerminalConfiguration.RSE)
ai_task.ai_channels.add_ai_voltage_chan("NI_DAQ/ai7", name_to_assign_to_channel="FlowSignal", min_val=0, max_val=5, terminal_config=nidaqmx.constants.TerminalConfiguration.RSE)


DOChannel(name=NI_DAQ/port1/line1)

In [ ]:
do_task = nidaqmx.Task()
#do_task.do_channels.add_do_chan("NI_DAQ/port1/line0")
#do_task.do_channels.add_do_chan("NI_DAQ/port1/line1")
#do_task.do_channels.add_do_chan("NI_DAQ/port1/line2")
print(do_task.channels.do_output_drive_type)
do_task.close()

DigitalDriveType.ACTIVE_DRIVE


In [42]:
#valve open
do_task.write([False, True], auto_start=True, timeout=3)

1

In [48]:
#valve close
do_task.write([True, False], auto_start=True, timeout=3)

1

In [45]:
#flow rate
ao_task.write(0, auto_start=True, timeout=3)

1

In [51]:
#[Valve test (valve open/close), flow signal output]
ai_task.read()

[0.2959593246341683, 0.6700669863494113]

In [52]:
ai_task.close()
ao_task.close()
do_task.close()

testing gate control with DAQ

In [1]:
import nidaqmx
import time

In [73]:
with nidaqmx.Task() as task:
    task.do_channels.add_do_chan("NI_DAQ/port0/line3")
    task.do_channels.add_do_chan("NI_DAQ/port0/line4")
    task.write([False, False], auto_start=True, timeout=3)
    task.write([False, True], auto_start=True, timeout=3)
    #task.write([True, False], auto_start=True, timeout=3)
    

In [2]:
with nidaqmx.Task() as task:
    task.do_channels.add_do_chan("NI_DAQ/port0/line0")
    task.do_channels.add_do_chan("NI_DAQ/port0/line1")
    task.write([False, False], auto_start=True, timeout=3)

In [67]:
import numpy as np

In [41]:
np.linspace(start=0, stop=10, num=10, endpoint=False)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [69]:
array = np.linspace(start=0, stop=10, num=10, endpoint=False)
for trial, pressure in enumerate(array):
    print(f"trial: {trial}\npressure: {pressure}")
    print(len(array))

trial: 0
pressure: 0.0
10
trial: 1
pressure: 1.0
10
trial: 2
pressure: 2.0
10
trial: 3
pressure: 3.0
10
trial: 4
pressure: 4.0
10
trial: 5
pressure: 5.0
10
trial: 6
pressure: 6.0
10
trial: 7
pressure: 7.0
10
trial: 8
pressure: 8.0
10
trial: 9
pressure: 9.0
10


In [25]:
#up
do_task = nidaqmx.Task()
do_task.do_channels.add_do_chan("NI_DAQ/port1/line2")
do_task.do_channels.add_do_chan("NI_DAQ/port1/line1")
do_task.write([True,False],auto_start=True, timeout=3)
time.sleep(0.5)
for x in range(400):
    do_task.write([True,False],auto_start=True,timeout=10)
    time.sleep(.0000025)
    do_task.write([False,False],auto_start=True,timeout=10)
    time.sleep(0.0000025)
do_task.close()

In [ ]:
#down
do_task = nidaqmx.Task()
do_task.do_channels.add_do_chan("NI_DAQ/port1/line2") #PUL
do_task.do_channels.add_do_chan("NI_DAQ/port1/line1") #Dir
do_task.write([True,False],auto_start=True, timeout=3)
time.sleep(.00001)
for x in range(400):
    do_task.write([True,True],auto_start=True,timeout=10)
    time.sleep(.0000025)
    do_task.write([False,True],auto_start=True,timeout=10)
    time.sleep(0.0000025)
do_task.close()

In [28]:
do_task = nidaqmx.Task()
do_task.do_channels.add_do_chan("NI_DAQ/port1/line2") #PUL
do_task.do_channels.add_do_chan("NI_DAQ/port1/line1") #Dir
do_task.write([True,True],auto_start=True, timeout=3)
time.sleep(30)
do_task.write([False,False],auto_start=True,timeout=10)
do_task.close()

In [47]:
import nidaqmx
import time

# NI USB-6002: PUL = port1/line2, DIR = port1/line1
do_task = nidaqmx.Task()
do_task.do_channels.add_do_chan("NI_DAQ/port1/line1")  # DIR
do_task.do_channels.add_do_chan("NI_DAQ/port1/line2")  # PUL
# Change direction here: True (down) or False (up)
direction = True  # Toggle this to reverse motor direction

# Set DIR before stepping
do_task.write([direction, False], auto_start=True)
time.sleep(0.00001)  # ≥5 µs DIR setup time

# Generate 400 step pulses (1.8°/step motor = 0.5 rev)
for _ in range(3200):
    do_task.write([direction, True], auto_start=True)   # PUL HIGH
    time.sleep(0.000005)
    do_task.write([direction, False], auto_start=True)  # PUL LOW
    time.sleep(0.000005)

do_task.close()

In [48]:
import pyvisa

In [58]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('USB0::0x05E6::0x6500::04386498::INSTR',
 'USB0::0xF4EC::0xEE38::SDSMMFCD4R9625::INSTR',
 'ASRL1::INSTR',
 'ASRL4::INSTR')

In [59]:
rm.list_resources_info()

{'USB0::0x05E6::0x6500::04386498::INSTR': ResourceInfo(interface_type=<InterfaceType.usb: 7>, interface_board_number=0, resource_class='INSTR', resource_name='USB0::0x05E6::0x6500::04386498::0::INSTR', alias='Digital_Multimeter'),
 'USB0::0xF4EC::0xEE38::SDSMMFCD4R9625::INSTR': ResourceInfo(interface_type=<InterfaceType.usb: 7>, interface_board_number=0, resource_class='INSTR', resource_name='USB0::0xF4EC::0xEE38::SDSMMFCD4R9625::0::INSTR', alias='Oscilloscope'),
 'ASRL1::INSTR': ResourceInfo(interface_type=<InterfaceType.asrl: 4>, interface_board_number=1, resource_class='INSTR', resource_name='ASRL1::INSTR', alias='CD_reader'),
 'ASRL4::INSTR': ResourceInfo(interface_type=<InterfaceType.asrl: 4>, interface_board_number=4, resource_class='INSTR', resource_name='ASRL4::INSTR', alias='Power_TR')}

In [60]:
osc = rm.open_resource("USB0::0xF4EC::0xEE38::SDSMMFCD4R9625::INSTR")

In [64]:
float(osc.query("C1:OFST?"))

0.0

In [65]:
osc.close()

In [74]:
trial=1
trial+=1
print(trial)

2
